<a href="https://colab.research.google.com/github/syedamiel/SocialMediaAnalysis/blob/main/Web_and_Social_Media_Analytics_MAS_TikTok(Part_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initial cleaning for TikTok comments

In [1]:

import json
from datetime import datetime
import pandas as pd

# Load the dataset
file_path = "/content/dataset_tiktok-comments-scraper_2025-03-28_01-13-55-183.json"

with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Extract relevant data
extracted_data = []
comment_count = 0

for comment in data:
    comment_info = {
        "video_url": comment.get("videoWebUrl", ""),
        "timestamp": comment.get("createTimeISO", ""),
        "username": comment.get("uniqueId", ""),
        "comment_text": comment.get("text", ""),
        "likes": comment.get("diggCount", 0),
        "replies": comment.get("replyCommentTotal", 0),
    }

    extracted_data.append(comment_info)
    comment_count += 1

print(f"Extracted {comment_count} comments.")

# Convert timestamps
for comment in extracted_data:
    if comment["timestamp"]:
        comment["timestamp"] = datetime.strptime(
            comment["timestamp"], "%Y-%m-%dT%H:%M:%S.%fZ"
        ).strftime("%Y-%m-%d %H:%M:%S")

# Prepare data for CSV export
csv_data = [
    [c["video_url"], c["timestamp"], c["username"], c["comment_text"], c["likes"], c["replies"]]
    for c in extracted_data
]

# Ensure all data is properly formatted (convert None values to empty strings)
csv_data_clean = [[str(item) if item is not None else "" for item in row] for row in csv_data]

# Create a DataFrame
df = pd.DataFrame(csv_data_clean, columns=["Video URL", "Timestamp", "Username", "Comment Text", "Likes", "Replies"])

# Debugging - Print sample data
print(df.head(10))
print(df.dtypes)

# Define the output file path
excel_file_path = "/content/cleaned_tiktok_comments.xlsx"

# Export to Excel
df.to_excel(excel_file_path, index=False)

print(f"Filtered data saved to: {excel_file_path}")


Extracted 612 comments.
                                           Video URL            Timestamp  \
0  https://www.tiktok.com/@malaysiaairlines/video...  2025-03-26 10:35:53   
1  https://www.tiktok.com/@malaysiaairlines/video...  2025-03-26 14:14:19   
2  https://www.tiktok.com/@malaysiaairlines/video...  2025-03-26 12:40:24   
3  https://www.tiktok.com/@malaysiaairlines/video...  2025-03-27 21:25:32   
4  https://www.tiktok.com/@malaysiaairlines/video...  2025-03-26 19:47:41   
5  https://www.tiktok.com/@malaysiaairlines/video...  2025-03-26 16:41:45   
6  https://www.tiktok.com/@malaysiaairlines/video...  2025-03-27 05:16:03   
7  https://www.tiktok.com/@malaysiaairlines/video...  2025-03-26 13:02:49   
8  https://www.tiktok.com/@malaysiaairlines/video...  2025-03-25 21:45:53   
9  https://www.tiktok.com/@malaysiaairlines/video...  2025-03-24 13:06:04   

            Username                                       Comment Text Likes  \
0        syafiq_.171                           

The below one works as well for the Posts/Videos

In [ ]:
import json
import pandas as pd

# Load the dataset
input_file_path = "/content/dataset_tiktok-profile-scraper_2025-03-27_08-56-41-895.json"
output_file_path = "/content/cleaned_tiktok_posts.xlsx"

with open(input_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Define the required keys and order
required_keys = ["id", "text", "createTimeISO", "webVideoUrl", "diggCount",
                 "shareCount", "playCount", "collectCount", "commentCount", "videoDuration"]

# Clean the dataset while maintaining order
cleaned_data = []
for entry in data:
    cleaned_entry = {key: entry.get(key, None) for key in required_keys if key != "videoDuration"}

    # Extract video duration from nested "videoMeta" dictionary
    cleaned_entry["videoDuration"] = entry.get("videoMeta", {}).get("duration", None)

    cleaned_data.append(cleaned_entry)

# Convert to DataFrame with the specified column order
df = pd.DataFrame(cleaned_data, columns=required_keys)

# Save the cleaned data to an Excel file
df.to_excel(output_file_path, index=False)

print(f"Cleaned data saved to: {output_file_path}")



Cleaned data saved to: /content/cleaned_tiktok_posts.xlsx


In [2]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=f389890c7e00d25710fcd854af6d9c13648fc4c5ad0c288d2c1210c0b5122c70
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


Merge Posts file and Comments file to match comments to a posts using Video URL

In [9]:
import pandas as pd
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Ensure consistent language detection results
DetectorFactory.seed = 0

# Load the TikTok posts dataset
posts_df = pd.read_excel("/content/cleaned_tiktok_posts.xlsx", engine="openpyxl")

# Load the TikTok comments dataset
comments_df = pd.read_excel("/content/cleaned_tiktok_comments.xlsx", engine="openpyxl")

# Display the first few rows to check the structure
print("Posts DataFrame:")
print(posts_df.head())

print("\nComments DataFrame:")
print(comments_df.head())

# Rename the 'videoWebUrl' column in posts_df to 'Video URL' for merging
posts_df = posts_df.rename(columns={"webVideoUrl": "Video URL"})

# Ensure both datasets have a 'URL' column for merging
if "Video URL" not in posts_df.columns or "Video URL" not in comments_df.columns:
    raise KeyError("Both datasets must contain a 'URL' column to merge.")

# Merge the datasets on the 'URL' column
merged_df = comments_df.merge(posts_df, on="Video URL", how="inner")

# Function to detect language (handling exceptions)
def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

# Apply language detection to the comments column
merged_df["language"] = merged_df["Comment Text"].astype(str).apply(detect_language)

# Keep only English comments
english_comments_df = merged_df[merged_df["language"] == "en"].drop(columns=["language"])

# Save the cleaned data to a new Excel file
output_filename = "cleaned-tiktok-data.xlsx"
english_comments_df.to_excel(output_filename, index=False, engine="openpyxl")

print(f"\nProcessing complete! Cleaned data saved as: {output_filename}")


Posts DataFrame:
                    id                                               text  \
0  7486021286582406407  Explore Surabaya with Malaysia Airlines 🌿\n\nD...   
1  7486057349388487943  Discover Singapore with Malaysia Airlines ✈️​\...   
2  7485333454524026129  Bonjour, Paris! Fly with Malaysia Airlines to ...   
3  7484970389886520592  Bonjour, Paris! \n\nLast night, we took off fr...   
4  7484291637737164053  We’re getting ready to take off to the City of...   

              createTimeISO  \
0  2025-03-26T13:00:00.000Z   
1  2025-03-26T10:05:54.000Z   
2  2025-03-24T13:00:00.000Z   
3  2025-03-23T11:48:25.000Z   
4  2025-03-22T02:00:00.000Z   

                                         webVideoUrl  diggCount  shareCount  \
0  https://www.tiktok.com/@malaysiaairlines/video...         54           1   
1  https://www.tiktok.com/@malaysiaairlines/video...         52           1   
2  https://www.tiktok.com/@malaysiaairlines/video...        223           3   
3  https://www.ti

Sentiment Analysis

In [10]:
!pip install pandas openpyxl textblob

In [11]:
import pandas as pd
from textblob import TextBlob

# Load the cleaned TikTok data
input_filename = "/content/cleaned-tiktok-data.xlsx"
df = pd.read_excel(input_filename, engine="openpyxl")

# Function to analyze sentiment
def get_sentiment(text):
    analysis = TextBlob(str(text))  # Ensure text is a string
    polarity = analysis.sentiment.polarity  # Get sentiment polarity

    # Classify sentiment based on polarity score
    if polarity > 0:
        return "Positive"
    elif polarity < 0:
        return "Negative"
    else:
        return "Neutral"

# Apply sentiment analysis to the 'Comment Text' column
df["Sentiment"] = df["Comment Text"].apply(get_sentiment)

# Save the updated dataset with sentiment analysis
output_filename = "tiktok-data-with-sentiment.xlsx"
df.to_excel(output_filename, index=False, engine="openpyxl")

print(f"\nSentiment analysis complete! Data saved as: {output_filename}")



Sentiment analysis complete! Data saved as: tiktok-data-with-sentiment.xlsx


Sentiment Analysis using vaderSentiment (less accurate)

In [12]:
!pip install pandas openpyxl vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.5 MB/s eta 0:00:00


In [13]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load the cleaned TikTok data
input_filename = "/content/cleaned-tiktok-data.xlsx"
df = pd.read_excel(input_filename, engine="openpyxl")

# Initialize VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to determine sentiment
def get_vader_sentiment(text):
    scores = analyzer.polarity_scores(str(text))  # Get sentiment scores
    compound_score = scores["compound"]  # Use compound score for classification

    # Classify based on compound score thresholds
    if compound_score >= 0.05:
        return "Positive"
    elif compound_score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

# Apply sentiment analysis to the 'Comment Text' column
df["Sentiment"] = df["Comment Text"].apply(get_vader_sentiment)

# Save the updated dataset with sentiment analysis
output_filename = "tiktok-data-with-vader-sentiment.xlsx"
df.to_excel(output_filename, index=False, engine="openpyxl")

print(f"\nSentiment analysis complete! Data saved as: {output_filename}")



Sentiment analysis complete! Data saved as: tiktok-data-with-vader-sentiment.xlsx
